## test-google-news-index

In [1]:
import os
os.chdir('..')
os.getcwd()

'/Users/apple/WRK/dcolinmorgan/dots'

In [2]:
from DOTS.scrape import get_OS_data, get_test_gnews
from DOTS.pull import pull_data, process_url
from DOTS.ingestion_utils import safe_iter_pull, iter_pull, reduce_newlines, scrape_selenium_headless

In [3]:
import getpass
import os

# os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import OpenSearchVectorSearch
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import CharacterTextSplitter

In [4]:
from dotenv import load_dotenv
import subprocess, json, argparse, os,requests
load_dotenv()
os_url = os.getenv('OS_TOKEN')
openai_KEY = os.getenv('OPENAI_API_KEY')

lobstr_key = os.getenv('LOBSTR_KEY')

In [5]:
response = get_test_gnews(1000)
hits = response["hits"]["hits"]
articles = pull_data(hits)
indices = [i for i, x in enumerate(articles) if len(str(x)) < 50]
print(len(indices))
# for i in indices:
#     url = hits[i]['_source']['metadata']['link']
#     # print(url)
#     try:
#         articles[i] = scrape_selenium_headless(url,browser='undetected_chrome')
#     except:
#         pass
# indices2 = [i for i, x in enumerate(articles) if len(str(x)) <50]
# print(len(indices2))

grabbing text from url:  43%|████▎     | 353/821 [00:41<00:51,  9.13it/s]/Users/apple/WRK/dcolinmorgan/dots/DOTS/pull.py:132: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(response.text, 'html.parser')
grabbing text from url: 100%|██████████| 821/821 [01:23<00:00,  9.80it/s]

194


In [6]:
indices2 = [i for i, x in enumerate(articles) if len(str(x)) <50]
print(len(indices2))
articles_not_in_indices2 = [article for i, article in enumerate(articles) if i not in indices2]

194


## g.feat

In [7]:
import pandas as pd
import os
os.getcwd()
import graphistry

from dotenv import load_dotenv
import subprocess, json, argparse, os,requests
load_dotenv()
g_pass = os.getenv('g_pass')
g_user = os.getenv('g_user')

graphistry.register(api=3,protocol="https", server="hub.graphistry.com", username=g_user, password=g_pass) ## key id, secret key
from graphistry.features import search_model, topic_model, ngrams_model, ModelDict, default_featurize_parameters, default_umap_parameters

In [8]:
indices2 = [i for i, x in enumerate(articles) if len(str(x)) <50]
(len(indices2))
articles_not_in_indices2 = [article for i, article in enumerate(articles) if i not in indices2]

df=pd.Series(articles_not_in_indices2).to_frame()
object_columns = df.select_dtypes(include=['object']).columns
df[object_columns] = df[object_columns].astype(str)
df.columns=['text']
df['text'] = df['text'].str.replace("', '", ' ')
df['text'] = df['text'].str.replace('", "', ' ')
df['text'] = df['text'].str.replace('" "', ' ')
df['text'] = df['text'].str.replace("' '", ' ')
df['text'] = df['text'].str.replace("\'", ' ')
df['text'] = df['text'].str.replace("]", ' ')
df['text'] = df['text'].str.replace("[", ' ')
object_columns = df.select_dtypes(include=['object']).columns
df[object_columns] = df[object_columns].astype(str)

In [9]:
df

,text
0,"""TEHRAN, Apr. 02 (MNA) – Russia s Federal Se..."
1,The Nigerian Army has confirmed a minor expl...
2,"Raipur, Apr 2 (PTI) In a major anti-insurge..."
3,Notifications can be managed in browser pref...
4,The police arrested a 19-year-old Rotterdam ...
...,...
622,Cloudy this evening with showers after midni...
623,"The latest breaking updates, delivered strai..."
624,"WEATHER ALERT Christian Terry, Digital Conte..."
625,"LINCOLN, Neb. (KOLN) - A man was cited after..."


In [10]:
%%time
g=graphistry.nodes(df)
g2 = g.umap(df,**topic_model)
g2 = g2.dbscan() #min_dist=1, min_samples=3)
g3 = g2.transform_dbscan(df,return_graph=False)
df2=pd.DataFrame(g2.get_matrix())

max_index_per_row = df2.idxmax(axis=1)
top_3_indices = max_index_per_row.value_counts().index[:3]
top_3_indices

* Ignoring target column of shape (627, 0) in UMAP fit, as it is not one dimensional

Index(['text: vatandaşlarımızın, nightclub, gündoğdu',
       'text: ljlo9b5ncj, h9zep9aikz, louisville',
       'text: northeasterly, thursday, snowfall'],
      dtype='object')

In [23]:
g2.plot()

## featurize

In [15]:
from DOTS.feat import featurize_stories
import graphistry
import umap

In [16]:
featurize_stories(str(articles[711]), top_k = 3, max_len=512)

['tuesday morning', 'morning', 'deterioration']

In [17]:
rank_articles=[]
from tqdm import tqdm
indices3 = [i for i in range(len(articles)) if i not in indices2]

for i in tqdm(indices3):
    try:
        cc = featurize_stories(str(articles[i]), top_k = 3, max_len=512)
        rank_articles.append([cc])
    except:
        pass

100%|██████████| 627/627 [14:54<00:00,  1.43s/it]


In [ ]:
os.getcwd()

In [18]:
flattened_list = [item for sublist in rank_articles for item in sublist]

In [ ]:
import pandas as pd
import os
os.getcwd()
import graphistry

from dotenv import load_dotenv
import subprocess, json, argparse, os,requests
load_dotenv()
g_pass = os.getenv('g_pass')
g_user = os.getenv('g_user')

graphistry.register(api=3,protocol="https", server="hub.graphistry.com", username=g_user, password=g_pass) ## key id, secret key

In [ ]:
# pd.DataFrame(flattened_list).to_csv('big_test.txt')
# data=pd.read_csv('../../../big_test.txt',index_col=0)

In [20]:
# import pandas as pd
data=pd.DataFrame(flattened_list)  # each ranked feature is a row
data=data.dropna(how='any')
data=data.drop_duplicates()

object_columns = data.select_dtypes(include=['object']).columns
data[object_columns] = data[object_columns].astype(str)

# g = graphistry.edges(data).materialize_nodes()
g = graphistry.nodes(data)
# g2 = g.featurize()
# g2._clustersummary()

g22 = g.umap()
g22=g22.dbscan()#min_dist=0.1, min_samples=3) #82 groups
g33 = g22.transform_dbscan(data,return_graph=False)
df22=pd.DataFrame(g22.get_matrix())

max_index_per_row = df22.idxmax(axis=1)
top_33_indices = max_index_per_row.value_counts().index[:3]
top_33_indices

Index(['2: weather, severe, swathes', '2: thunderstorms, storms, storm',
       '1: firefighters, firefighter, daughters'],
      dtype='object')

In [24]:
g22.plot() # .encode_point_color('_dbscan',palette=["hotpink", "dodgerblue"],as_continuous=True).plot()